In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import datetime
from selenium.common.exceptions import TimeoutException


In [2]:
#functions
def get_info_by_text(soup, target_text):
    try:
        elements = soup.find_all("div", class_='row row--dense')
        for element in elements:
            spec_name_element = element.find("div", class_='spec-name col-sm-3 col-5')
            if spec_name_element and target_text in spec_name_element.text:
                info_value = element.find("span", class_='mr-1 mb-1').text.strip()
                return info_value
    except (AttributeError, IndexError):
        pass
    return ""
def get_info_by_text2(soup, target_text):
    try:
        elements = soup.find_all("div", class_='row row--dense')[0]
        spec_name_elements = elements.find_all("div", class_='spec-name col-sm-3 col-5')

        for spec_name_element in spec_name_elements:
            if target_text in spec_name_element.text:
                info_element = spec_name_element.find_next("div", class_='col-sm-9 col-7')
                info_value = info_element.text.strip() if info_element else ""
                return info_value
    except (AttributeError, IndexError):
        pass
    return ""

def get_title(soup):
    try:
        title = soup.find("h1", class_='text-h5 text-capitalize')
        title_value = title.text
        title_string = title_value.strip()
    except (IndexError, AttributeError):
        title_string = ""
    return title_string
def get_price(soup):
    try:
        price = soup.find("span", dir='ltr')
        price_value = price.text
        price_string = price_value.strip()
    except (IndexError, AttributeError):
        price_string = ""
    return price_string
def get_city(soup):
    try:
        city = soup.find("div",class_='mt-6').find("div", class_='v-card v-card--flat v-sheet theme--light')
        city_value = city.text
        city_string = city_value.strip()
    except (IndexError, AttributeError):
        city_string = ""
    return city_string
def get_date(soup):
    return get_info_by_text2(soup, 'Date')

def get_views(soup):
    return get_info_by_text2(soup, 'Vues')
def get_gearbox(soup):
    return get_info_by_text(soup, 'Boite')

def get_version(soup):
    return get_info_by_text(soup, 'Finition')

def get_model(soup):
    return get_info_by_text(soup, 'Modèle')

def get_brand(soup):
    return get_info_by_text(soup, 'Marque')

def get_papiers(soup):
    return get_info_by_text(soup, 'Papiers')

def get_energy(soup):
    return get_info_by_text(soup, 'Energie')

def get_color(soup):
    return get_info_by_text(soup, 'Couleur')

def get_engine(soup):
    return get_info_by_text(soup, 'Moteur')

def get_km(soup):
    return get_info_by_text(soup, 'Kilométrage')

def get_year(soup):
    return get_info_by_text(soup, 'Année')
def getdata(url):
    # Set up Chrome options
    proxy_server = "http://105.104.3.29:3128"
    
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--window-size=1920,1200')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-gpu')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-infobars')
    options.add_argument(f'--proxy-server={proxy_server}')
    
    # Create a WebDriver instance with the specified options
    driver = webdriver.Chrome(options=options)
    
    try:
        # Navigate to the target URL
        driver.get(url)
    
        # Your scraping logic goes here
        html_content = driver.page_source
    
        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(html_content, "lxml")
    
        # Print the title and parsed content
        # print(driver.title)
        #print(soup)
    
    except NoSuchElementException as e:
        print(f"Element not found: {e}")
    
    finally:
        # Close the browser
        driver.quit()

    return soup

In [3]:
import csv
import pandas as pd
import numpy as np
import time  # Import the time module for adding delays

i = 390
url = "https://www.ouedkniss.com/automobiles/390"

# Open the CSV file in append mode
with open("car_data_final.csv", mode='a', newline='', encoding='utf-8') as file:
    # Create a CSV writer object
    csv_writer = csv.writer(file)

    while i <= 500:
        try:
            print(i)
            HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
                       'Accept-Language': 'en-US, en;q=0.5'}

            soup = getdata(url)

            # Fetch links as List of Tag Objects
            links = soup.find_all("a", class_='d-flex flex-column flex-grow-1 v-card v-card--link v-sheet o-announ-card-content pb-1 theme--light')

            # Loop for extracting product details from each link
            for link in links:
                try:
                    new_soup = getdata("https://www.ouedkniss.com" + link.get('href'))

                    # Function calls to display all necessary product information
                    row = [
                        get_title(new_soup),
                        get_price(new_soup),
                        get_date(new_soup),
                        get_views(new_soup),
                        get_color(new_soup),
                        get_gearbox(new_soup),
                        get_energy(new_soup),
                        get_engine(new_soup),
                        get_brand(new_soup),
                        get_model(new_soup),
                        get_version(new_soup),
                        get_year(new_soup),
                        get_km(new_soup),
                        get_city(new_soup)
                    ]

                    # Write the row to the CSV file
                    csv_writer.writerow(row)

                except Exception as inner_exception:
                    print(f"Inner Error: {inner_exception}. Skipping this entry.")

            url = "https://www.ouedkniss.com/automobiles/" + str(i + 1)
            
            i += 1

        except Exception as outer_exception:
            # Print the error and the last successful value of i
            print(f"Outer Error: {outer_exception}. Last successful i: {i - 1}")

            # Introduce a delay before retrying (you can adjust the duration)
            time.sleep(5)  # 5 seconds delay


390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
